faiss事前準備:

miniconda_python(3.10版本):

https://docs.conda.io/projects/miniconda/en/latest/miniconda-other-installer-links.html

裝完打開anaconda prompt:

做一個環境```conda create -n VScode-env-3.10.13 python=3.10.13```

做完啟動```conda activate VScode-env-3.10.13```

安裝faiss插件

```conda install -c pytorch faiss-cpu=1.7.4 mkl=2021 blas=1.0=mkl```

```conda install -c pytorch/label/nightly faiss-cpu```

關掉prompt 重新開啟VScode 右上角kernel 選擇another kernel->python env->VScode-env-3.10.13

VScode商城:

python

jupyter

import和拿key

沒裝openai_SDK使用:```pip install openai```

沒裝tabulate使用:```pip install tabulate```

沒裝pandas:```pip install pandas```

沒設定key使用:```setx OPENAI_API_KEY “<yourkey>”```

```echo %OPENAI_API_KEY%```可以看你的KEY

In [1]:
import os
from openai import OpenAI
client = OpenAI()
OpenAI.api_key = os.getenv('OPENAI_API_KEY')

# input在此輸入

In [2]:
input = "10+11 = ?"

# instruction在此輸入

In [3]:
instruction = "You are customer service from Flora. You must answer questions related to Flora. You must answer based on the history questions and answers, otherwise you just leave only a special mark:'*'"

先準備faiss

讀embeddings資料

並把dataframe轉為numpy

In [4]:
import pandas as pd
import numpy as np

csv_file = 'data\embeddingsV3_3072.csv'
df = pd.read_csv(csv_file)
data = df.to_numpy()
df.head(5)

C:\Users\USER\AppData\Local\Temp\ipykernel_13348\2915898147.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,0,1,2,3,4,5,6,7,8,9,...,3062,3063,3064,3065,3066,3067,3068,3069,3070,3071
0,-0.030886,-0.020970,0.002210,-0.032358,-0.014908,-0.007581,0.005008,0.012017,-0.034472,0.018240,...,0.001131,0.019792,-0.011917,0.007220,-0.018026,0.003841,0.001450,0.020649,-0.003747,0.021291
1,0.013026,0.013851,-0.009579,-0.059783,0.008046,-0.015229,0.002933,0.004625,-0.002641,0.046384,...,0.017729,0.013825,0.001612,-0.019378,0.011802,-0.009483,-0.006945,0.005122,0.004068,0.009393
2,-0.005118,-0.024766,-0.014762,-0.029704,-0.016605,-0.011532,-0.001856,0.039089,0.005199,0.034809,...,0.013769,0.028311,0.001599,-0.015600,0.024998,-0.024624,-0.000467,-0.005411,0.007058,0.008451
3,-0.019629,0.002125,-0.002155,-0.013240,0.015541,-0.034832,-0.044084,0.013315,0.035132,0.020191,...,0.003723,0.032956,-0.020729,-0.005295,0.016266,0.004657,-0.020029,0.008870,0.011921,0.033432
4,0.005265,0.015462,0.002796,-0.057215,0.013133,-0.032900,-0.001634,0.008408,-0.005457,0.024303,...,0.020498,0.016450,0.004387,-0.020389,-0.004006,0.007847,0.003600,0.005015,0.010267,0.010481


確認內容

In [5]:
print(data.shape[0])
print(type(data))
print(data.dtype)
print(data[0][0])

669
<class 'numpy.ndarray'>
float64
-0.0308859776705503


寫入維度 並把data加入faiss的index

In [6]:
import faiss

index = faiss.IndexFlatL2(data.shape[1])
index.add(data)

做問題的embedding

In [7]:
response = client.embeddings.create(input=input, model="text-embedding-3-large", dimensions=3072)
question = np.array(response.data[0].embedding, ndmin=2)
print(question)

[[-0.03381647 -0.00795166 -0.0120192  ... -0.01242596  0.00014555
  -0.01282474]]


TOP-k的設定並查找問題

In [8]:
k = 3

distances, indices = index.search(question, k)

拿答案

In [9]:
csv_question = 'data\manual.csv'
qdf = pd.read_csv(csv_question)
qarray = qdf.to_numpy()

看答案和距離

In [10]:
answer = [(qarray[i], float(dist)) for dist, i in zip(distances[0], indices[0])]
print(answer)

[(array(['How many people can join one room at once? ',
       'Currently, there is no limit to the number of people to join a Flora room.'],
      dtype=object), 1.4905953407287598), (array([' How many stops are there on the map?',
       'The Grand Tour contains 27 stops, including 2 introductory stops (5 & 10) and abundant indigenous plants!'],
      dtype=object), 1.5109682083129883), (array([' I was wondering: What is the maximum amount of people that are allowed to join a Flora room?',
       'Currently, there is no limit to the number of people to join a Flora room.'],
      dtype=object), 1.5349600315093994)]


距離設定0.25內才顯示

# Prompt內容在此

In [11]:
history = ""
min_distance = 0.25
Qcount = 0
for i in range(0,k):
    if answer[i][1]<min_distance:
        Qcount+=1
        history = history +str(i+1)+". "+ answer[i][0][0]+"\n"+answer[i][0][1]+"\n"
prompt = "Here is "+str(Qcount)+" history questions and answers related to this question according to Flora's database\n"+history+"Here is my question:\n"+input
print(prompt)

Here is 0 history questions and answers related to this question according to Flora's database
Here is my question:
10+11 = ?


創assistant,thread,message!

In [12]:
from tabulate import tabulate

assistant = client.beta.assistants.create(
    name="Flora's Customer Service",
    instructions=instruction,
    model="gpt-3.5-turbo-1106"
)

thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content= prompt
)


沒裝tabulate很醜

tabulate文件:https://pypi.org/project/tabulate/

表示assistant,thread,message內容

In [13]:
print(tabulate(assistant,headers=['assistant','content'],tablefmt="simple_outline"))

┌──────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ assistant    │ content                                                                                                                                                                                       │
├──────────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ id           │ asst_vhjBnZNaEjCgbcMdSQBcE2Ap                                                                                                                                                                 │
│ created_at   │ 1710842795                                                                                                                                         

In [14]:
print(tabulate(thread,headers=['thread','content'],tablefmt="simple_outline"))

┌────────────┬─────────────────────────────────┐
│ thread     │ content                         │
├────────────┼─────────────────────────────────┤
│ id         │ thread_FusGcxQ0w6zKRMy5pdn1WLNr │
│ created_at │ 1710842795                      │
│ metadata   │ {}                              │
│ object     │ thread                          │
└────────────┴─────────────────────────────────┘


In [15]:
print(tabulate(message,headers=['message','content'],tablefmt="simple_outline"))

┌────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ message            │ content                                                                                                                                                                                             │
├────────────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ id                 │ msg_rNc3Cs793kFguwWr3oVQ3EaK                                                                                                                                                                        │
│ assistant_id       │                                                                                              

這次的run

可以使用新instruction複寫assisstant裡的instruction!

like:

```
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)
```


In [16]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

查看run的狀態!

以上如果還沒跑完會顯示queued!

以上如果跑完會顯示completed!

In [17]:
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
print(tabulate(run,headers=['run','content'],tablefmt='simple_outline'))

┌─────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ run             │ content                                                                                                                                                                                       │
├─────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ id              │ run_FfeNnaJMMAn6XNKCI70czBey                                                                                                                                                                  │
│ assistant_id    │ asst_vhjBnZNaEjCgbcMdSQBcE2Ap                                                                                                       

跑完後打開查看history messages!

In [18]:
import time
time.sleep(5)
messages = client.beta.threads.messages.list(
  thread_id=thread.id,
  order='asc'
)
for i in messages.data:
    print(tabulate(i,tablefmt='simple_outline'))

┌────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ id                 │ msg_rNc3Cs793kFguwWr3oVQ3EaK                                                                                                                                                                        │
│ assistant_id       │                                                                                                                                                                                                     │
│ completed_at       │                                                                                                                                                                                                     │
│ content            │ [TextContentBlock(text=Text(annotations=[], value="Here is 0 history questions and answers re

訊息顯示

In [19]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id,
  order='asc'
)
for i in messages.data:
  print(i.content[0].text.value)
  print("\n\n\n")

Here is 0 history questions and answers related to this question according to Flora's database
Here is my question:
10+11 = ?




*




